In [8]:

import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

path="data/coqa-train.json"


with open(path,"r")as f:
    train=json.load(f)
    
question_0=collections.Counter()
question_1=collections.Counter()
qa_list=[]
total=0

for paragraph in tqdm(train["data"]):
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"]
        question=tokenize(question_text)
        answer_text=answer_dict["input_text"]
        qa_list.append((question_text,answer_text))
        total+=1
        
        if len(question)>0:
            question_0[question[0]]+=1
            if len(question)>1:
                question_1[" ".join(question[0:2])]+=1

        


100%|██████████| 7199/7199 [00:11<00:00, 624.55it/s]


In [18]:
print(*[(q[0],q[1]/total) for q in question_0.most_common()[0:20]],sep="\n")
print(*[(q[0],q[1]/total) for q in question_1.most_common()[0:20]],sep="\n")

('What', 0.23897576555266137)
('Who', 0.11518035472677571)
('How', 0.07971688127605916)
('Did', 0.05545482157813837)
('what', 0.054847349673713956)
('Where', 0.053024933960440696)
('Was', 0.03746997155926993)
('When', 0.033300505306175045)
('who', 0.02869844542417186)
('Is', 0.025835964177565878)
('Why', 0.021307537253674746)
('how', 0.02056200355279023)
('Does', 0.016567415575211464)
('where', 0.013125074783473082)
('And', 0.013106666543945069)
('Which', 0.012683277034800777)
('did', 0.012259887525656484)
('was', 0.009535468075510598)
('when', 0.008210074829493682)
('Were', 0.007667031763417305)
('What did', 0.04188794904599299)
('What was', 0.03865730300882675)
('What is', 0.034883613905584136)
('How many', 0.028689241304407854)
('Who was', 0.025458595267241618)
('Who is', 0.01787440058170037)
('Did he', 0.01754305227019614)
('Where did', 0.014643754544534133)
('What does', 0.013502443693797344)
('When did', 0.01338279013686526)
('Who did', 0.010971310758695592)
('Where was', 0.01005

In [2]:
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

In [21]:
print(*[(q,a) for q,a in qa_list[0:50]],sep="\n")
print(*[(q.split()[0],a) for q,a in qa_list[0:50]],sep="\n")

('When was the Vat formally opened?', 'It was formally established in 1475')
('what is the library for?', 'research')
('for what subjects?', 'history, and law')
('and?', 'philosophy, science and theology')
('what was started in 2014?', 'a  project')
('how do scholars divide the library?', 'into periods')
('how many?', 'five')
('what is the official name of the Vat?', 'The Vatican Apostolic Library')
('where is it?', 'in Vatican City')
('how many printed books does it contain?', '1.1 million')
('when were the Secret Archives moved from the rest of the library?', 'at the beginning of the 17th century;')
('how many items are in this secret collection?', '150,000')
('Can anyone use this library?', 'anyone who can document their qualifications and research needs.')
('what must be requested to view?', 'unknown')
('what must be requested in person or by mail?', 'Photocopies')
('of what books?', 'only books published between 1801 and 1990')
('What is the Vat the library of?', 'the Holy See')
(

In [28]:
%%bash
head -c 100000 "data/coqa-train.json"

{
  "version": "1.0",
  "data": [
    {
      "source": "wikipedia",
      "id": "3zotghdk5ibi9cex97fepx7jetpso7",
      "filename": "Vatican_Library.txt",
      "story": "The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-y

In [32]:
text=json.dumps(train["data"][0],indent=4)
print(text)

{
    "source": "wikipedia",
    "id": "3zotghdk5ibi9cex97fepx7jetpso7",
    "filename": "Vatican_Library.txt",
    "story": "The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of ma

In [50]:
import json



with open("data/coqa-dev-v1.0.json","r")as f:
    data=json.load(f)
    
text=json.dumps(data,indent=4)
print(text[:50000])

{
    "version": "1.0",
    "data": [
        {
            "source": "mctest",
            "id": "3dr23u6we5exclen4th8uq9rb42tel",
            "filename": "mc160.test.41",
            "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n\"What are you doing, C

In [51]:
import json



with open("data/coqa-dev-rule.json","r")as f:
    data=json.load(f)
    
text=json.dumps(data,indent=4)
print(text[0:50000])

{
    "version": "1.0",
    "data": [
        {
            "source": "mctest",
            "id": "3dr23u6we5exclen4th8uq9rb42tel",
            "filename": "mc160.test.41",
            "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n\"What are you doing, C

In [22]:
with open("data/coqa-src-train-corenlp.txt","r")as f:
    data=json.load(f)
    
text=json.dumps(data,indent=4)
print(text[0:10000])

[
    {
        "question_interro": "when ?",
        "neg_interro": "was the vat formally opened ?",
        "vb_check": true
    },
    {
        "question_interro": "what ?",
        "neg_interro": "is the library for ?",
        "vb_check": true
    },
    {
        "question_interro": "what ?",
        "neg_interro": "for subjects ?",
        "vb_check": false
    },
    {
        "question_interro": "none_tag",
        "neg_interro": "and?",
        "vb_check": false
    },
    {
        "question_interro": "what ?",
        "neg_interro": "was started in 2014 ?",
        "vb_check": true
    },
    {
        "question_interro": "how ?",
        "neg_interro": "do scholars divide the library ?",
        "vb_check": true
    },
    {
        "question_interro": "how ?",
        "neg_interro": "many ?",
        "vb_check": false
    },
    {
        "question_interro": "what ?",
        "neg_interro": "is the official name of the vat ?",
        "vb_check": true
    },
    {
      

In [21]:
count=sum([1 if d["question_interro"]=="none_tag" else 0 for d in data])
print(count,len(data))
    

2703 108647


In [29]:
for i in range(3000):
    if i<3:
        continue
    d=data[i]
    his=data[i-1]
    his2=data[i-2]
    if d["vb_check"]==False:
        print(d)
        print(his)
        print(his2)
        print()

{'question_interro': 'none_tag', 'neg_interro': 'and?', 'vb_check': False}
{'question_interro': 'what ?', 'neg_interro': 'for subjects ?', 'vb_check': False}
{'question_interro': 'what ?', 'neg_interro': 'is the library for ?', 'vb_check': True}

{'question_interro': 'how ?', 'neg_interro': 'many ?', 'vb_check': False}
{'question_interro': 'how ?', 'neg_interro': 'do scholars divide the library ?', 'vb_check': True}
{'question_interro': 'what ?', 'neg_interro': 'was started in 2014 ?', 'vb_check': True}

{'question_interro': 'what ?', 'neg_interro': 'of books ?', 'vb_check': False}
{'question_interro': 'what ?', 'neg_interro': 'must be requested in person or by mail ?', 'vb_check': True}
{'question_interro': 'what ?', 'neg_interro': 'must be requested to view ?', 'vb_check': True}

{'question_interro': 'which ?', 'neg_interro': '?', 'vb_check': False}
{'question_interro': 'how many ?', 'neg_interro': 'remain ?', 'vb_check': True}
{'question_interro': 'are ?', 'neg_interro': 'all of the

In [30]:
from pycorenlp import StanfordCoreNLP
from tqdm import tqdm

import matplotlib.pyplot as plt
import networkx as nx


#corenlpを用いて疑問詞とその周辺を取り出す
#ex. How many icecream did you eat? -? How many icecream
#疑問詞を特定できないもの(interro_listの中のタグがあわられないもの)は除去


class CoreNLP():
    def __init__(self):
        self.nlp=StanfordCoreNLP('http://localhost:9000')
        self.interro_list=["WDT","WP"," WP$","WRB","VB","VBD","VBG","VBN","VBP","VBZ"]
        self.count=-1

    def forward(self,text):#input:(batch,seq_len)
        self.count+=1
        q=self.nlp.annotate(text, properties={'annotators': 'tokenize,ssplit,parse','outputFormat': 'json'})

        tokens=q["sentences"][0]["tokens"]
        deps=q["sentences"][0]["basicDependencies"]

        token_list=[]
        token_list.append({"index":0,"text":"ROOT"})
        interro_id=-1
        for token in tokens:
            token_list.append({"index":token["index"],"text":token["originalText"],"pos":token["pos"]})
        for token in tokens:
            if interro_id==-1 and token["pos"] in self.interro_list[0:4]:#疑問詞のチェック
                interro_id=token["index"]
        for token in tokens:
            if interro_id==-1 and token["pos"] in self.interro_list[4:]:#疑問詞のチェック
                interro_id=token["index"]

        #疑問詞がなかった時のエラー処理
        if interro_id==-1:
            print(text)
            #print(self.count)
            return "none_tag","none_tag"


        G = nx.DiGraph()
        G.add_nodes_from(range(len(token_list)))
        for dep in deps:
            G.add_path([dep["dependent"],dep["governor"]])
        if nx.has_path(G,interro_id,0)==False:
            print("error")
        s_path=nx.shortest_path(G,interro_id,0)


        if len(s_path)==2:#疑問詞だけ
            node_list=[s_path[0]]
        else:#疑問詞周り
            node_list=[node for node in G.nodes() if nx.has_path(G,node,s_path[-3])]
        neg_node_list=[node for node in G.nodes() if node not in node_list and node!=0]
        question=" ".join([token_list[node]["text"] for node in node_list])
        neg_question=" ".join([token_list[node]["text"] for node in neg_node_list])
        return question,neg_question

    def verb_check(self,text):
        q=self.nlp.annotate(text, properties={'annotators': 'tokenize,ssplit,pos','outputFormat': 'json'})
        pos_list=[]
        for sentence in q["sentences"]:
            for token in sentence["tokens"]:
                pos_list.append(token["pos"])

        for pos in pos_list:
            if "VB" in pos:
                return True
        return False

    def forward_verbcheck(self,text):#input:(batch,seq_len)
        self.count+=1
        q=self.nlp.annotate(text, properties={'annotators': 'tokenize,ssplit,parse','outputFormat': 'json'})

        tokens=q["sentences"][0]["tokens"]
        deps=q["sentences"][0]["basicDependencies"]

        token_list=[]
        token_list.append({"index":0,"text":"ROOT"})
        interro_id=-1
        for token in tokens:
            token_list.append({"index":token["index"],"text":token["originalText"],"pos":token["pos"]})
        for token in tokens:
            if interro_id==-1 and token["pos"] in self.interro_list[0:4]:#疑問詞のチェック
                interro_id=token["index"]
        for token in tokens:
            if interro_id==-1 and token["pos"] in self.interro_list[4:]:#疑問詞のチェック
                interro_id=token["index"]

        #疑問詞がなかった時のエラー処理
        if interro_id==-1:
            #print(self.count)
            return "none_tag",text,False

        #vb
        pos_list=[]
        for sentence in q["sentences"]:
            for token in sentence["tokens"]:
                pos_list.append(token["pos"])

        vb_check=False

        for pos in pos_list:
            if "VB" in pos:
                 vb_check=True

        G = nx.DiGraph()
        G.add_nodes_from(range(len(token_list)))
        for dep in deps:
            G.add_path([dep["dependent"],dep["governor"]])
        if nx.has_path(G,interro_id,0)==False:
            print("error")
        s_path=nx.shortest_path(G,interro_id,0)


        if len(s_path)==2:#疑問詞だけ
            node_list=[s_path[0]]
        else:#疑問詞周り
            node_list=[node for node in G.nodes() if nx.has_path(G,node,s_path[-3])]
        neg_node_list=[node for node in G.nodes() if node not in node_list and node!=0]
        question=" ".join([token_list[node]["text"] for node in node_list])
        neg_question=" ".join([token_list[node]["text"] for node in neg_node_list])
        return question,neg_question,vb_check


In [40]:
corenlp=CoreNLP()
corenlp.forward_verbcheck("Why?")

('Why', '?', False)

In [23]:
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

def c2wpointer(context_text,context,answer_start,answer_end):#answer_start,endをchara単位からword単位へ変換
    #nltk.tokenizeを使って分割
    #ダブルクオテーションがなぜか変化するので処理
    token_id={}
    cur_id=0
    for i,token in enumerate(context):
        start=context_text.find(token,cur_id)
        token_id[i]=(start,start+len(token))
        cur_id=start+len(token)
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_start and answer_start<=token_id[i][1]:
            answer_start_w=i
            break
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_end and answer_end<=token_id[i][1]:
            answer_end_w=i
            break
    return answer_start_w,answer_end_w

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

def test_process():
    
    dev=pd.read_json("data/coqa-dev.json")
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    ids=[]
    word2count=collections.Counter()
    char2count=collections.Counter()

    for i in tqdm(range(len(dev))):

        paragraph=dev.iloc[i][0]
        context_text=paragraph["story"]
        context=tokenize(context_text)

        for word in context:
            word2count[word]+=1
            for char in word:
                char2count[char]+=1

        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]

            question_text=question_dict["input_text"]
            question=tokenize(question_text)    
            answer_text=answer_dict["input_text"]
    

            for word in question:
                word2count[word]+=1
                for char in word:
                    char2count[char]+=1
                    
            contexts.append(context)
            questions.append(question)
            #answer_start=a["answer_start"]
            #answer_end=a["answer_start"]+len(a["text"])
            #answer_start,answer_end=c2wpointer(context_text,context,answer_start,answer_end)
            #answer_starts.append(answer_start)
            #answer_ends.append(answer_end)
            answer_texts.append(answer_text)
            
    print(len(answer_texts),answer_texts[0:100])

In [24]:
test_process()

100%|██████████| 500/500 [00:02<00:00, 191.75it/s]

7983 ['white', 'in a barn', 'no', 'with her mommy and 5 sisters', 'orange and white', 'no', 'she painted herself', 'the farmer', 'they started laughing', 'a bucket of water', 'licked her face', 'no', 'Asta.', 'a bottle', 'Asta.', 'Yes', 'Yes', 'a note', 'No', "Asta's papa", 'unknown', 'yes', 'unknown', 'An elderly Chinese lady and a little boy', 'Yes', 'a paper carrier bag', 'Yes', 'Nicole', 'Shanghai', 'mother', 'food', 'Yes', 'I am having heart surgery soon, so her mother has decided I need more nutrients', 'an iPad', 'hot soup and a container with rice, vegetables and either chicken, meat or shrimp, sometimes with a kind of pancake', 'I am now working on some more Chinese words', 'Yes', '"Thank you"', 'Yes.', 'Dennis Farina', 'Actor', 'No', 'Yes', 'No', 'Farina was cast in a film', 'Michael Mann', '"Thief"', 'cops or gangsters', 'He joined a TV show cast.', '"Law & Order"', 'Detective Joe Fontana', 'No', 'An expensive car', 'No', 'Flashy', 'No', 'No', 'A cop', 'school', 'No', "go to

In [8]:
from nltk.tokenize import word_tokenize,sent_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def cos_sim(v1, v2):
    if np.linalg.norm(v1)==0 or np.linalg.norm(v2)==0:
        return np.zeros(1)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def tf_idf_vec(paragraph):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(paragraph).toarray()
    return X

def tf_idf(paragraph,question,num_canditate):
    sentences=sent_tokenize(paragraph)
    sentences.append(question)
    vec=tf_idf_vec(sentences)
    cos={i:cos_sim(v,vec[-1]) for i,v in enumerate(vec[:-1])}
    cos=sorted(cos.items(),key=lambda x:-x[1])
    pred_question=" ".join([sentences[c[0]] for c in cos[0:num_canditate]])
    return cos[0][0:num_canditate]


In [87]:
#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections

from func.tf_idf import tf_idf

def head_find(tgt):
    q_head=["what","how","who","when","which","where","why","whose","whom","is","are","was","were","do","did","does"]
    tgt_tokens=word_tokenize(tgt)
    true_head="<none>"
    for h in q_head:
        if h in tgt_tokens:
            true_head=h
            break
    return true_head

def modify(sentence,question_interro):
    #head=head_find(question)
    """
    if answer in sentence:
        sentence=sentence.replace(answer," ans_rep_tag ")
    """
    #sentence=" ".join([sentence,"ans_pos_tag",answer,"interro_tag",question_interro])
    sentence=" ".join([sentence,"interro_tag",question_interro])
    return sentence

def modify_history(history,now):
    #head=head_find(question)
    """
    if answer in sentence:
        sentence=sentence.replace(answer," ans_rep_tag ")
    """
    #sentence=" ".join([sentence,"ans_pos_tag",answer,"interro_tag",question_interro])
    sentence=" ".join([history,"history_append_tag",now])
    return sentence

def history_maker(neg_interro,question_interro):
    interro_list=["what","where","who","why","which","whom","how",""]
    while True:
        index=random.randrange(len(interro_list))
        if interro_list[index]!=question_interro.split()[0]:
            break
    question=interro_list[index]+" "+neg_interro
    return question


def c2wpointer(context_text,context,answer_start,answer_end):#answer_start,endをchara単位からword単位へ変換
    #nltk.tokenizeを使って分割
    #ダブルクオテーションがなぜか変化するので処理
    token_id={}
    cur_id=0
    for i,token in enumerate(context):
        start=context_text.find(token,cur_id)
        token_id[i]=(start,start+len(token))
        cur_id=start+len(token)
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_start and answer_start<=token_id[i][1]:
            answer_start_w=i
            break
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_end and answer_end<=token_id[i][1]:
            answer_end_w=i
            break
    return answer_start_w,answer_end_w

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in word_tokenize(sent)]

#context_textを文分割して、answer_start~answer_end(char単位)のスパンが含まれる文を返す
#やってることはc2iと多分同じアルゴリズム
def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    sent_start_id=-1
    sent_end_id=-1
    start_id_list=[context_text.find(sent) for sent in context]
    end_id_list=[start_id_list[i+1] if i+1!=len(context) else len(context_text) for i,sent in enumerate(context)]
    for i,sent in enumerate(context):
        start_id=start_id_list[i]
        end_id=end_id_list[i]
        if start_id<=answer_start and answer_start<=end_id:
            sent_start_id=i
        if start_id<=answer_end and answer_end<=end_id:
            sent_end_id=i



    if sent_start_id==-1 or sent_end_id==-1:
        #sys.exit(-1)
        print("error")
        #sys.exit(-1)
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return sent_start_id,answer_sent


def data_process(input_path,src_path,tgt_path,dict_path):
    with open(input_path,"r") as f:
        data=json.load(f)
    with open(dict_path,"r") as f:
        corenlp_data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    ccount=0
    count=0
    answer_count=0
    context_len=[]
    for paragraph in data["data"]:

        context_text=paragraph["story"].lower()
        question_history=[]
        context_len.append(len(sent_tokenize(context_text)))
        
        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]
            question_text=question_dict["input_text"].lower()
            answer_text=answer_dict["input_text"].lower()
            question_history.append((question_text,answer_text))

            span_start=answer_dict["span_start"]
            span_end=answer_dict["span_end"]
            span_text=answer_dict["span_text"]
            turn_id=paragraph["questions"][i]["turn_id"]

            if span_start!=-1:
                sentence,start_id=answer_find(context_text,span_start,span_end,answer_replace)
                if len(question_history)==1:
                    continue
                a_text=" ".join([question_history[-2][0],question_history[-1][0]])
                #print(a_text)
               # tf_id=tf_idf(context_text," ".join(question_history[-2:]),num_canditate=1) 
                tf_id=tf_idf(context_text,a_text,num_canditate=1) 
                if start_id in tf_id:
                    answer_count+=1
                count+=1
            ccount+=1
            



    print(answer_count,count,answer_count/count)
    print(sum(context_len)/len(context_len))


#main
version="1.1"
type="split2-interro"
question_modify=True
question_interro=True


data_process(input_path="data/coqa-dev-v1.0.json",
            src_path="data/coqa-src-dev-{}.txt".format(type),
            tgt_path="data/coqa-tgt-dev-{}.txt".format(type),
            dict_path="data/coqa-dev-corenlp.json"
            )


KeyboardInterrupt: 




 10%|█         | 52/500 [00:23<03:22,  2.21it/s]

In [101]:
from nltk.tokenize import word_tokenize,sent_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def cos_sim(v1, v2):
    if np.linalg.norm(v1)==0 or np.linalg.norm(v2)==0:
        return np.zeros(1)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def tf_idf_vec(paragraph):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(paragraph).toarray()
    return X

def tf_idf(paragraph,question,num_canditate):
    sentences=sent_tokenize(paragraph)
    sentences.append(question)
    vec=tf_idf_vec(sentences)
    cos={i:cos_sim(v,vec[-1]) for i,v in enumerate(vec[:-1])}
    cos=sorted(cos.items(),key=lambda x:-x[1])
    pred_question=" ".join([sentences[c[0]] for c in cos[0:num_canditate]])
    return pred_question,[c[0] for c in cos[0:num_canditate]]

In [131]:
#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections

def head_find(tgt):
    q_head=["what","how","who","when","which","where","why","whose","whom","is","are","was","were","do","did","does"]
    tgt_tokens=word_tokenize(tgt)
    true_head="<none>"
    for h in q_head:
        if h in tgt_tokens:
            true_head=h
            break
    return true_head

def modify(sentence,question_interro):
    #head=head_find(question)
    """
    if answer in sentence:
        sentence=sentence.replace(answer," ans_rep_tag ")
    """
    #sentence=" ".join([sentence,"ans_pos_tag",answer,"interro_tag",question_interro])
    sentence=" ".join([sentence,"interro_tag",question_interro])
    return sentence

def modify_history(history,now):
    #head=head_find(question)
    """
    if answer in sentence:
        sentence=sentence.replace(answer," ans_rep_tag ")
    """
    #sentence=" ".join([sentence,"ans_pos_tag",answer,"interro_tag",question_interro])
    sentence=" ".join([history,"history_append_tag",now])
    return sentence

def history_maker(neg_interro,question_interro):
    interro_list=["what","where","who","why","which","whom","how",""]
    while True:
        index=random.randrange(len(interro_list))
        if interro_list[index]!=question_interro.split()[0]:
            break
    question=interro_list[index]+" "+neg_interro
    return question


def c2wpointer(context_text,context,answer_start,answer_end):#answer_start,endをchara単位からword単位へ変換
    #nltk.tokenizeを使って分割
    #ダブルクオテーションがなぜか変化するので処理
    token_id={}
    cur_id=0
    for i,token in enumerate(context):
        start=context_text.find(token,cur_id)
        token_id[i]=(start,start+len(token))
        cur_id=start+len(token)
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_start and answer_start<=token_id[i][1]:
            answer_start_w=i
            break
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_end and answer_end<=token_id[i][1]:
            answer_end_w=i
            break
    return answer_start_w,answer_end_w

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in word_tokenize(sent)]

#context_textを文分割して、answer_start~answer_end(char単位)のスパンが含まれる文を返す
#やってることはc2iと多分同じアルゴリズム
def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    sent_start_id=-1
    sent_end_id=-1
    start_id_list=[context_text.find(sent) for sent in context]
    end_id_list=[start_id_list[i+1] if i+1!=len(context) else len(context_text) for i,sent in enumerate(context)]
    for i,sent in enumerate(context):
        start_id=start_id_list[i]
        end_id=end_id_list[i]
        if start_id<=answer_start and answer_start<=end_id:
            sent_start_id=i
        if start_id<=answer_end and answer_end<=end_id:
            sent_end_id=i

    if sent_start_id==-1 or sent_end_id==-1:
        #sys.exit(-1)
        print("error")
        #sys.exit(-1)
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return sent_start_id,answer_sent


def data_process(input_path,src_path,tgt_path,dict_path):
    with open(input_path,"r") as f:
        data=json.load(f)
    with open(dict_path,"r") as f:
        corenlp_data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    ccount=0
    count=-1
    answer_count=[]
    context_len=[]

    for paragraph in data["data"][0:]:

        context_text=paragraph["story"].lower()
        question_history=[]
        cs=sent_tokenize(context_text)

        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]
            question_text=question_dict["input_text"].lower()
            answer_text=answer_dict["input_text"].lower()
            question_history.append((question_text,answer_text))

            span_start=answer_dict["span_start"]
            span_end=answer_dict["span_end"]
            span_text=answer_dict["span_text"]
            turn_id=paragraph["questions"][i]["turn_id"]
            
            count+=1
            d=corenlp_data[count]
            if d["vb_check"]==False and d["question_interro"]!="none_tag":
                if span_start!=-1:
                    start_id,sentence=answer_find(context_text,span_start,span_end,answer_replace)
                    if len(question_history)<=1:
                        continue
                        print("s")
                        a_text=question_history[-1][0]
                    else:
                        a_text=" ".join([question_history[-2][0],question_history[-2][1],question_history[-1][0]])
                    if question_history[-1][0]!=question_text:
                        print("eeeee")
                    ans_sentence,tf_id=tf_idf(context_text,a_text,num_canditate=1) 
                    """
                    print(*cs,sep="\n")
                    print()
                    print([question_history[-2][0],question_history[-2][1],question_history[-1][0]])
                    print(start_id)
                    print(tf_id)
                    print(answer_dict["span_text"])
                    print()
                    """
                    #tf_id=tf_idf(context_text,a_text,num_canditate=1) 
                    answer_count.append(start_id in tf_id)

        
    print(len(answer_count))
    print(np.average(answer_count),np.sum(answer_count))
    print(answer_count[0:30])
    print(sum(answer_count[0:100]),sum(answer_count[100:200]),sum(answer_count[400:600]),sum(answer_count[600:]))



#main
version="1.1"
type="split2-interro"
question_modify=True
question_interro=True


data_process(input_path="data/coqa-dev-v1.0.json",
            src_path="data/coqa-src-dev-{}.txt".format(type),
            tgt_path="data/coqa-tgt-dev-{}.txt".format(type),
            dict_path="data/coqa-dev-corenlp.json"
            )


871
0.5177956371986223 451
[False, False, False, True, True, True, True, True, False, True, True, True, False, True, False, True, True, False, True, False, False, False, True, True, False, False, False, True, True, False]
54 64 103 139


In [69]:


#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections

with open("data/coqa-dev-rule.json")as f:
    data=json.load(f)

question_rule_list=[]
count=0
modify_count=0
for paragraph in tqdm(data["data"]):
    context_text=paragraph["story"].lower()
    question_history=[]
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"].lower()
        question_rule_list.append(question_text)


100%|██████████| 500/500 [00:00<00:00, 64599.31it/s]


In [92]:
from pycorenlp import StanfordCoreNLP
from tqdm import tqdm

import matplotlib.pyplot as plt
import networkx as nx
import json

nlp=StanfordCoreNLP('http://localhost:9000')

In [108]:
text="Who funds medlineplus?"
q=nlp.annotate(text, properties={'annotators': 'tokenize,ssplit,parse','outputFormat': 'json'})
p=q["sentences"][0]["parse"]
print(p)
#q=json.dumps(q,indent=4)
#print(q)

(ROOT
  (FRAG
    (SBARQ
      (WHNP
        (WHNP (WP Who))
        (NP
          (NP (NNS funds))
          (NP (NN medlineplus))))
      (. ?))))


In [113]:
c0=[]
c1=[]

with open("data/pred_coqa-dev-interro.txt")as f:
    for line in f:
        c0.append(line.rstrip())

with open("data/pred_coqa-dev-interro-cand1.txt")as f:
    for line in f:
        c1.append(line.rstrip())
        
ans_count=[1 if c0[i]==c1[i] else 0 for i in range(len(c0))]
print(np.average(ans_count),len(ans_count))
    

0.5119453924914675 879


In [118]:
c0=[]
c1=[]

with open("data/coqa-src-dev-interro.txt")as f:
    for line in f:
        c0.append(line.rstrip())

with open("data/coqa-src-dev-interro_cand1.txt")as f:
    for line in f:
        c1.append(line.rstrip())
        
ans_count=[1 if c0[i]==c1[i] else 0 for i in range(len(c0))]
print(np.average(ans_count),len(ans_count),np.sum(ans_count))
print(ans_count[0:30])

0.502844141069397 879 442
[0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]
